In [2]:
"""
是按照时间保存的，怎么同步呢？
实在不行，先读取一遍bag，把点云保存成dataframe，再同时返回毫米波和激光雷达数据
"""
import rospy
import rosbag
from sensor_msgs.msg import PointCloud2
from sensor_msgs import point_cloud2
import numpy as np
import pickle

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [3]:
p1 = np.array([0, 2])
p2 = np.array([2, 2])
p3 = np.array([1, 0])

p2_p1 = p2 - p1
p3_p1 = p3 - p1
if np.cross(p2_p1, p3_p1) < 0:
    p3, p2 = p2, p3


In [5]:
points = np.array([[1, 1], [0, 1], [2, 1]])

In [10]:
points - p1

array([[ 1, -1],
       [ 0, -1],
       [ 2, -1]])

In [11]:
p2_p1 = p2 - p1
p3_p2 = p3 - p2
p1_p3 = p1 - p3
mask1 = np.cross(p2_p1, points-p1)

In [12]:
p2_p1, mask1

(array([ 1, -2]), array([ 1, -1,  3]))

In [5]:
bag = rosbag.Bag("/home/dingrong/Code/ackermann_car_nav/data/20230530/floor31_h1_120_L_120_angle_30_param1_2023-05-30-15-58-38.bag")
bag_data = bag.read_messages(topics=['/laser_point_cloud', '/mmwave_radar_point_cloud'])


In [6]:
laser_list, mmwave_list = [], []
for topic, msg, t in bag_data:
    if topic == '/laser_point_cloud':
        points = point_cloud2.read_points_list(
            msg, field_names=['x', 'y']
        )
        x_pos = [p.x for p in points]
        y_pos = [p.y for p in points]
        point_cloud = np.array([x_pos, y_pos]).T
        laser_list.append((t.to_sec(), point_cloud))
    if topic == '/mmwave_radar_point_cloud':
        points = point_cloud2.read_points_list(
            msg, field_names=['x', 'y', 'z', 'vel']
        )
        x_pos = [p.x for p in points]
        y_pos = [p.y for p in points]
        z_pos = [p.z for p in points]
        vel = [p.vel for p in points]
        point_cloud = np.array([x_pos, y_pos, z_pos, vel]).T
        mmwave_list.append((t.to_sec(), point_cloud))

In [7]:
n = len(laser_list)
m = len(mmwave_list)
n, m

(0, 139)

In [5]:
laser_list[0][0] < mmwave_list[0][0]

True

In [6]:

min_idx, min_v = 0, np.inf
for i in range(n-m+1):
    diff = np.abs(laser_list[i][0] - mmwave_list[0][0])
    if diff < min_v:
        min_v = diff
        min_idx = i

laser_list = laser_list[min_idx:]
all_point_cloud = []
for i in range(m):
    tmp = (laser_list[i][0], laser_list[i][1], mmwave_list[i][1])
    all_point_cloud.append(tmp)

In [7]:
with open("/home/dingrong/Code/ackermann_car_nav/data/ransac_static2_2023-05-21-20-31-07.pkl", 'wb') as f:
    pickle.dump(all_point_cloud, f)